# Optimisation dans les graphes - Code d'optimisation d'un aéroport (Cas de Strasbourg-Entzheim)

Léo le Douarec, Antoine Reilhes.

Vous trouverez dans la suite 4 celllules de code à exécuter
1. Importation des données
2. Affichage du graphe d'intervalles modélisant l'aéroport de Strasbourg-Entzheim (non colorié, puis colorié) et du dictionnaire de coloration associé (en `print`)
3. Gestion des retards (affichage des vols qui poseraient problème en cas de retard, et valeur à partir de laquelle un retard entraîne une augmentation du nombre chromatique $\chi(G)$
4. recherche d'intervalles candidats pour proposer des créneaux pour de nouvelles lignes aériennes de compagnies désireuses d'ajouter des liaisons vers l'aéroport, sans augmenter $\chi(G)$ (première méthode, dont les résultats sont présentés dans le rapport).

## Import des données

In [1]:
import networkx as nx
import pandas as pd
import time
import datetime

import matplotlib.pyplot as plt #pour afficher les graphes
import matplotlib.colors as mpl #pour afficher les graphes coloriés


strasbourg_data = pd.read_excel("aeroport_strasbourg.xlsx")#, dtype={"Départ":datetime.time, "Arrivée":datetime.time}) #pas nécessaire, le type datetime est reconnu d'office
#print(strasbourg_data)

## Coloration du graphe d'intervalles définissant l'aéroport de Strasbourg-Entzheim

In [2]:

def hours_to_minutes(date):
    """
    attention: cette fonction ne traite pas encore les données manquantes 
    (celles où date = NaN).
    """
    #print(date) #pour debug
    date_string = date.strftime('%H:%M')
    hours = int(date_string[0:2])
    minutes = int(date_string[3:5])
    return(60*hours + minutes)
    
## test
#print(hours_to_minutes(datetime.time(0, 0)))

#conversion des données "time" en minutes (on part de minuit = 0 minutes), en utilisant la fonction précédente
strasbourg_data["Départ"] = strasbourg_data["Départ"].apply(hours_to_minutes)
##################### Prise en compte du retard moyen (à décommenter si on veut le prendre en compte) ####################################
"""retard_moyen = 27 # source = https://airportinfo.live/fr/statistiques-aeroport/sxb-strasbourg-international-de-strasbourg
strasbourg_data["Départ"] += retard_moyen""" 
#print(strasbourg_data["Départ"]) #test
strasbourg_data["Arrivée"] = strasbourg_data["Arrivée"].apply(hours_to_minutes)

nb_lignes = len(strasbourg_data.index)
intervals_strasbourg = [(strasbourg_data.loc[i,"Arrivée"], strasbourg_data.loc[i,"Départ"]) for i in range(nb_lignes)] #création des intervalles pour chaque avion

G = nx.interval_graph(intervals_strasbourg) #graphe_strasbourg

## Visualisation du Graphe
print("\nNoeuds du Graphe (bornes de chaque intervalle, en minutes) calculées par la fonction nx.interval_graph")
print(f"{sorted(G.nodes)}\n")
nx.draw_networkx(
     G,
     with_labels=False,
     node_size=50,
     edge_color="grey",
     font_size=12,
     font_color="#333333",
     width=2,
)
plt.title("Graphe d'intervalle représentant l'aéroport de Strasbourg")

plt.show()

def strategy_left_sort(G,colors): #stratégie de coloriage: par ordre croissant d'heure d'arrivée ! Primordial pour assurer l'optimalité dans le cas général.
    return sorted(list(G.nodes), key = lambda l : l[0]) 
## Coloration du graphe
coloriage = nx.greedy_color(G,strategy=strategy_left_sort)

unique_colors = set(coloriage.values())

graph_color_to_mpl_color = dict(zip(unique_colors, mpl.TABLEAU_COLORS))
node_colors = [graph_color_to_mpl_color[coloriage[n]] for n in G.nodes()]

pos = nx.spring_layout(G, seed=14)
nx.draw_networkx(
     G,
     pos,
     with_labels=False,
     node_size=50,
     node_color=node_colors,
     edge_color="grey",
     font_size=12,
     font_color="#333333",
     width=2,
)
plt.title("Graphe de l'aéroport de Strasbourg, colorié")
plt.show()

print(f"Dictionnaire avec les couleurs trouvées par coloration gloutonne: \n {coloriage}")


Noeuds du Graphe (bornes de chaque intervalle, en minutes) calculées par la fonction nx.interval_graph
[(0, 400), (0, 405), (0, 420), (490, 520), (500, 530), (525, 555), (560, 645), (600, 1105), (645, 670), (760, 790), (850, 890), (855, 905), (960, 995), (1055, 1085), (1070, 1120), (1120, 1150), (1220, 1255), (1225, 1255), (1285, 1315), (1305, 1439), (1395, 1439), (1410, 1439)]

Dictionnaire avec les couleurs trouvées par coloration gloutonne: 
 {(0, 400): 0, (0, 405): 1, (0, 420): 2, (490, 520): 0, (500, 530): 1, (525, 555): 0, (560, 645): 0, (600, 1105): 1, (645, 670): 2, (760, 790): 0, (850, 890): 0, (855, 905): 2, (960, 995): 0, (1055, 1085): 0, (1070, 1120): 2, (1120, 1150): 0, (1220, 1255): 0, (1225, 1255): 1, (1285, 1315): 0, (1305, 1439): 1, (1395, 1439): 0, (1410, 1439): 2}


## Recherche de créneaux (intervalles) candidats pour ajouter de nouveaux vols

In [3]:
composantes_connexes = list(nx.connected_components(G))


inter = []
for l in composantes_connexes:
    #on définit les bornes de l'intersection 
    a = min(l, key= lambda x : x[0])[0] # plus petit horaire d'arrivée (borne inférieure de l'intervalle qu'on va renvoyer)
    d = max(l, key=lambda x: x[1])[1] # plus grand horaire de départ (borne supérieure)

    #puis des variables utiles pour définir le ou les intervalle(s) que l'on va renvoyer
    plus_grd_arrivee = max(l, key= lambda x : x[0])[0] #plus grand horaire d'arrivée
    plus_petit_depart = min(l, key= lambda x : x[1])[1]

    if plus_grd_arrivee < plus_petit_depart: # cas où l'intersection est non vide: plus grand horaire d'arrivée < plus petit horaire de départ. 

        if plus_grd_arrivee == a: #si l'intervalle à gauche de l'intersection est un singleton, on ne le garde pas.
            inter.append([(plus_petit_depart,d)])

        elif plus_petit_depart == d: #si l'intervalle à droite est un singleton, on ne le garde pas.
            inter.append([(a,plus_grd_arrivee)])
        else: 
            inter.append([(a,plus_grd_arrivee),(plus_petit_depart,d)]) #on renvoie deux intervalles: l'un qui est "à gauche de l'intersection", et l'autre qui est à droite.
    
    

print(f"Intervalles candidats pour ajouter des avions: {inter}") #chaque sous-liste dans {inter} correspond aux deux intervalles trouvés dans chaque composante connexe.

Intervalles candidats pour ajouter des avions: [[(400, 420)], [(1220, 1225)]]


Remarques: 
- Dans la liste ci-dessus, chaque sous-liste représente les intervalles candidats trouvés au sein d'une même composante connexe.
- les singletons ne sont pas intéressants, on les ignore (c'est à cela que servent le 2e `if`, et le `elif`).
- D'autres pistes pour améliorer et augmenter le nombre de résultats ont été présentées pendant la présentation orale, et dans le rapport.

## Etude de l'impact de retards ponctuels
La cellule suivante met quelques secondes à s'exécuter.

In [4]:
# gestion des retards :
retards =[]
avions = []

for j in range(len(strasbourg_data["Arrivée"])):
    for k in range(200):
        # décalage de l'intervalle dû au retard
        strasbourg_data.at[j,"Arrivée"] = strasbourg_data.at[j,"Arrivée"] + k
        strasbourg_data.at[j,"Départ"]  = strasbourg_data.at[j,"Départ"] + k
        

        intervals_strasbourg = [(strasbourg_data.loc[i,"Arrivée"], strasbourg_data.loc[i,"Départ"]) for i in range(23)]

        G = nx.interval_graph(intervals_strasbourg) #graphe_strasbourg

        # coloriage du graphe alteré par le retard
        coloriage = nx.greedy_color(G, strategy=strategy_left_sort)
        unique_colors = set(coloriage.values())

        if len(unique_colors) == 4: # si le retard surcharge l'aéroport
            retards.append(k) # temps de retard 
            avions.append(j) # avions en question

        strasbourg_data.at[j,"Arrivée"] = strasbourg_data.at[j,"Arrivée"] - k
        strasbourg_data.at[j,"Départ"]  = strasbourg_data.at[j,"Départ"] - k
print(f"Durée de retard à partir de laquelle le nombre chromatique augmente: {min(retards)}")
print(f"Ensemble des avions dont les retards surchargent l'aéroport: {set(avions)}")

Durée de retard à partir de laquelle le nombre chromatique augmente: 65
Ensemble des avions dont les retards surchargent l'aéroport: {3, 4, 5, 6, 9, 10, 11, 12, 13, 17, 18, 19}


Les résultats ont été analysés au sein du rapport et lors de la présentation orale. N'hésitez pas à revenir vers nous si besoin.

Léo le Douarec et Antoine Reilhes.